# SHAP

In [ ]:
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import shap
import matplotlib.pyplot as plt

# Load model & tokenizer
model_path = './bert-ft'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()

# Example inputs
texts = ["im getting on borderlands 2 and i will murder you me all"]

# Prediction function for SHAP
def model_predict(x):
    if isinstance(x, np.ndarray):
        
        x = x.astype(str).tolist()
    elif isinstance(x, str):
        x = [x]
    elif not isinstance(x, list):
        x = list(x)

    # Tokenize -> model -> softmax probabilities
    enc = tokenizer(x, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        out = model(**enc)
        probs = torch.softmax(out.logits, dim=-1).cpu().numpy()
    return probs

masker = shap.maskers.Text(tokenizer=r"\W+")

explainer = shap.Explainer(model_predict, masker)

# Compute SHAP values
shap_values = explainer(texts)

# Plotting
shap.plots.text(shap_values[0])


In [ ]:
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import shap
import matplotlib.pyplot as plt

# Load model & tokenizer
model_path = './bert-ft'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()

# Example inputs
texts = ["The biggest disappointment of my life came a year ago."]

# Prediction function for SHAP
def model_predict(x):
    if isinstance(x, np.ndarray):
        x = x.astype(str).tolist()
    elif isinstance(x, str):
        x = [x]
    elif not isinstance(x, list):
        x = list(x)

    enc = tokenizer(x, padding=True, truncation=True, return_tensors="pt")
    
    input_ids = enc['input_ids'].long()
    with torch.no_grad():
        out = model(input_ids=input_ids)
        probs = torch.softmax(out.logits, dim=-1).cpu().numpy()
    return probs

# SHAP explainer for text
masker = shap.maskers.Text(tokenizer=r"\W+")

explainer = shap.Explainer(model_predict, masker)

# Compute SHAP values
shap_values = explainer(texts)

# Plotting
shap.plots.text(shap_values[0])

plt.close()

# LIME

In [9]:
import random
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from lime.lime_text import LimeTextExplainer

# Load the saved model and tokenizer
model_path = './bert-ft'  # Ensure this is your correct path
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # Set the model to evaluation mode

# Load the dataset
dataset_path = 'twitter-sentiment/sentiment-dataset.csv'  # Replace with your dataset path
dataset = pd.read_csv(dataset_path)

# Randomly select a sample from the dataset
random_sample = random.choice(dataset['comment'].tolist())

# Tokenize the input text
inputs = tokenizer([random_sample], padding=True, truncation=True, return_tensors="pt")

# Define a prediction function for LIME
def model_predict(texts):
    # Tokenize the input text into the format BERT understands
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    
    # Make predictions
    with torch.no_grad():
        outputs = model(**encodings)
    
    # Return the softmax probabilities for LIME
    return torch.nn.functional.softmax(outputs.logits, dim=-1).numpy()

# Initialize the LIME explainer
explainer = LimeTextExplainer(class_names=['Negative', 'Neutral', 'Positive', 'Irrelevant'])  # Adjust class names if needed

# Explain the randomly selected sample
explanation = explainer.explain_instance(random_sample, model_predict, num_features=10)

# For non-notebook environments, save the explanation as an HTML file and open it
html_exp = explanation.as_html()
with open("lime_explanation3.html", "w") as f:
    f.write(html_exp)

print("Explanation saved as 'lime_explanation3.html'. Open this file in your browser to view the explanation.")


Explanation saved as 'lime_explanation3.html'. Open this file in your browser to view the explanation.
